In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [92]:
### Scrape All ID

In [93]:
url = 'https://www.oktatas.hu/kozneveles/intezmenykereso/intezmenyek_a_szamok_tukreben/erettsegi_statisztikak/!DARI_ErettsegiStat/oh.php?id=kir_erett_talalat&oldal='
head = {'Cookie' : 'wcmid=user#21976512; DARI_ErettsegiStat-sessionid=uj5a81vqh5ll064p9tncv32tl2; Mainappstate-DARI_ErettsegiStat=%2Foh.php; DARI_Intezmenykereso-sessionid=c4vvvhg6g9vivaupgir6b0f695; Mainappstate-DARI_Intezmenykereso=%2Foh.php; Mainappstate-KIR_Intezmenykereso=%2FIntezmeny%2FIndex%2F035218'}
OMs = []
pages = 23
for i in range(pages):
    res = requests.get(url + str(i+1), headers=head)
    soup = BeautifulSoup(res.content)
    OMs.extend([tr.find_all('td')[1].text for tr in soup.find_all('tr')[2:-1]])
    print(f'The {i+1}. page Done!')

The 1. page Done!
The 2. page Done!
The 3. page Done!
The 4. page Done!
The 5. page Done!
The 6. page Done!
The 7. page Done!
The 8. page Done!
The 9. page Done!
The 10. page Done!
The 11. page Done!
The 12. page Done!
The 13. page Done!
The 14. page Done!
The 15. page Done!
The 16. page Done!
The 17. page Done!
The 18. page Done!
The 19. page Done!
The 20. page Done!
The 21. page Done!
The 22. page Done!
The 23. page Done!


In [95]:
df = pd.DataFrame(data=OMs)

In [1]:
df[df.duplicated()].unique()

NameError: name 'df' is not defined

In [403]:
pd.DataFrame(data=OMs)

,0
0,027240
1,027264
2,027396
3,027397
4,027398
...,...
1115,203408
1116,203419
1117,203420
1118,203421


In [102]:
## Test on Szilagyi

In [56]:
omid = '035219'

In [57]:
res = requests.get('https://www.oktatas.hu/kozneveles/intezmenykereso/koznevelesi_intezmenykereso/!DARI_Intezmenykereso/oh.php?id=kir_int_mod&int_omkod=' + omid)

In [58]:
soup = BeautifulSoup(res.content)

In [59]:
#Fotablazat
html_admin = str(soup.findAll('table')[2])
df_admin = pd.read_html(html_admin, index_col=1)[0].drop(columns=0).rename(columns={2 : 'value'})

In [60]:
## Data gather from admin table: 

#school_id = df_admin.loc['Az intézmény OM azonosítója:'][2]
#school_name = df_admin.loc['Az intézmény megnevezése:'][2]
#school_dean = df_admin.loc['Az intézmény vezetője:'][2]
#school_dean_title = df_admin.loc['Beosztása:'].iloc[0][2]
#school_address = df_admin.loc['Címe:'][2]
#school_type = df_admin.loc['Ellátott feladatok:'][2].split(',')

In [61]:
#### Feladatellatasi hely

In [62]:
html_sub_schools = soup.find(id='adat_felhely').findAll('table')[1::2]

In [63]:
len(html_sub_schools)

1

In [64]:
df_subs = pd.concat(pd.read_html(str(html_sub_schools))).drop(columns=5).rename(columns={0:'sub_id',1 : 'sub_school', 2 : 'school_address', 3 : 'school_country', 4 : 'school_type'}).reset_index(drop=True)

In [65]:
#### Basic Stats on Children
html_basics = str(soup.find('div', id='adata01t01').find('table'))
html_teachers = str(soup.find('div', id='adata01t06').find('table'))
html_students = str(soup.find('div', id='adata04t18').find('table'))



In [66]:
df_basics = pd.read_html(html_basics,header=[0,1,2], index_col=0)[0]

In [67]:
# Students
df_teachers = pd.read_html(html_teachers, index_col=[1,2])[0].drop(columns=[0,4]).rename(columns={3 : 'count'})

In [68]:
df_student = pd.read_html(html_students, index_col=0, header=[0,1,2])[0]

In [69]:
### Competency Test - get the average of all recent results
html_iq = str(soup.find('div', id='adat_okm2019').find('table'))

In [70]:
df_iq  = pd.read_html(html_iq, header=1)[0].drop(columns='Unnamed: 0')

In [71]:
#Get only One feladat ellatasi hely
#df_iq[df_iq['Telephely megnevezése és címe'].str.contains(school_address)]

In [72]:
### Enrollment data 2020 - need more? maybe
html_enroll2020 = str(soup.find('div', id='adat_kifir2020').find('table'))

In [73]:
df_enroll2020 = pd.read_html(html_enroll2020, header=0)[0].drop(columns='Unnamed: 0')

In [74]:
#Erettsegi

html_2020 = soup.find('div', id='adat_stat2020_vtargy').findAll('table')[0::2]

In [75]:
len(html_2020) ### give you all table

1

In [76]:
## by concat we create one unified table 

df_erett2020 = pd.concat(pd.read_html(str(html_2020), header=0)).drop(columns='Unnamed: 0').rename(columns={'Unnamed: 1' : 'Telephely megnevezése és címe'})

In [82]:
df_collection = {
    'omid' : omid,
    'sub_school_len' : len(html_sub_schools),
    'erettsegi_table_len' : len(html_2020),
    'admin' : html_admin,
    'sub_schools' : html_sub_schools,
    'basic' : html_basics,
    'students' : html_students,
    'teachers' : html_teachers,
    'iq_2019' : html_iq,
    'erettsegi_2020' : html_2020,
    'enroll_2020' : html_enroll2020
    
}

In [83]:
pd.DataFrame(df_collection)

,omid,sub_school_len,erettsegi_table_len,admin,sub_schools,basic,students,teachers,iq_2019,erettsegi_2020,enroll_2020
0,035219,1,1,"<table align=""center"" border=""0"" cellpadding=""...","[\n, [\n, [001], \n, [<a class=""slide"" id=""001...","<table align=""center"" border=""0"" cellpadding=""...","<table align=""center"" border=""0"" cellpadding=""...","<table align=""center"" border=""0"" cellpadding=""...","<table align=""center"" border=""0"" cellpadding=""...","[\n, [\n, [ ], \n, [ ], \n, [érettségi vizsgat...","<table align=""center"" border=""0"" cellpadding=""..."


In [85]:
df_collection.keys()

dict_keys(['omid', 'sub_school_len', 'erettsegi_table_len', 'admin', 'sub_schools', 'basic', 'students', 'teachers', 'iq_2019', 'erettsegi_2020', 'enroll_2020'])

In [70]:
################################################################################

#### Basic Quantitative Data - Summerize for all subschools

In [45]:
#### Basic Stats on Children
t_basics = soup.find('div', id='adata01t01').find('table')
t_teachers = soup.find('div', id='adata01t06').find('table')
t_students = soup.find('div', id='adata04t18').find('table')


In [56]:

df_basics

Összes feladatel-  látási helyek száma  \
          Összes feladatel-  látási helyek száma   
          Összes feladatel-  látási helyek száma   
Gimnázium                                      1   
Összesen                                       1   

          Gyermekek, tanulók nyitó létszáma          \
                                   Összesen   ebből   
                                   Összesen leányok   
Gimnázium                               501     256   
Összesen                                501     256   

                                                              \
                                                               
          gyógypeda-gógiai nevelésben, oktatásban résztvevők   
Gimnázium                                                  3   
Összesen                                                   3   

                                          \
                                           
          felnőttok-tatásban részt vevők   
Gimnázium                              0   
Összesen                               0   

          Fő munkaviszony keretében pedagógus, oktató munkakörben alkalmazottak nyitóállománya  \
                                                                                      Összesen   
                                                                                      Összesen   
Gimnázium                                                 51                                     
Összesen                                                  51                                     

                    Osztályterem, szaktanterem / csoportszoba összesen  \
          ebből nők Osztályterem, szaktanterem / csoportszoba összesen   
          ebből nők Osztályterem, szaktanterem / csoportszoba összesen   
Gimnázium        36                                                 21   
Összesen         36                                                 21   

          Osztályok, csoportok száma összesen  \
          Osztályok, csoportok száma összesen   
          Osztályok, csoportok száma összesen   
Gimnázium                                  16   
Összesen                                   16   

          Nevelő és oktató munkát közvetlenül segítők, egyéb munkakörben dolgozók nyitóállománya  \
                                                                                        Összesen   
                                                                                        Összesen   
Gimnázium                                                 16                                       
Összesen                                                  16                                       

                     
          Ebből nők  
          Ebből nők  
Gimnázium        10  
Összesen         10

##### Basics

In [295]:
df_basics  = pd.read_html(str(t_basics),header=0, index_col=0)[0]

no_hs_student = df_basics['Gyermekek, tanulók nyitó létszáma'].loc['Gimnázium']  
no_hs_student_female = df_basics['Gyermekek, tanulók nyitó létszáma.1'].loc['Gimnázium']  
no_rooms = df_basics['Osztályterem, szaktanterem / csoportszoba összesen'].loc['Gimnázium']
no_classes= df_basics['Osztályok, csoportok száma összesen'].loc['Gimnázium']  

##### Students

In [296]:
df_students = pd.read_html(str(t_students), index_col=[1,2])[0]

### tons of important info ignored here
no_fail = df_students.loc[('évfolyamismétlő','évfolyamismétlő')][3]

##### Teachers

In [297]:
df_teachers = pd.read_html(str(t_teachers), index_col=0, header=0)[0]
fulltime_teachers = df_teachers['Összesen'].loc['Teljes munkaidős']
parttime_teachers = df_teachers['Összesen'].loc['Részmunkaidős']
total_teachers = df_teachers['Összesen'].loc['Összesen']

### Competency test

In [298]:
### Competency Test - get the average of all recent results

t_iq = soup.find('div', id='adat_okm2019').find('table')
df_competency = pd.read_html(str(t_iq),index_col=3, header=1)[0]

math_competency = df_competency['Matematika'].mean()
lit_competency = df_competency['Szövegértés'].mean()

### Erettsegi test

In [299]:
### Erettsegi Tables - GSCI

t_2018= soup.find('div', id='adat_stat2018_vtargy').find('table')
t_2019= soup.find('div', id='adat_stat2019_vtargy').find('table')
t_2020= soup.find('div', id='adat_stat2020_vtargy').find('table')

In [300]:
df_erett_2020 = pd.read_html(str(t_2020), header=0)[0]

In [301]:
hu_gcse = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='magyar nyelv és irodalom') & (df_erett_2020['szint']=='közép')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
hu_gcse_pop,hu_gcse_perc,hu_gcse_avg = hu_gcse[0] if len(hu_gcse)==1 else [None,None,None]

hu_A = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='magyar nyelv és irodalom') & (df_erett_2020['szint']=='emelt')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
hu_A_pop,hu_A_perc,hu_A_avg = hu_A[0] if len(hu_A)==1 else [None,None,None]

math_gcse = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='matematika') & (df_erett_2020['szint']=='közép')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
math_gcse_pop,math_gcse_perc,math_gcse_avg = math_gcse[0] if len(math_gcse)==1 else [None,None,None]

math_A = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='matematika') & (df_erett_2020['szint']=='emelt')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
math_A_pop,math_A_perc,math_A_avg = math_A[0] if len(math_A)==1 else [None,None,None]

hist_gcse = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='történelem') & (df_erett_2020['szint']=='közép')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
hist_gcse_pop,hist_gcse_perc,hist_gcse_avg = hist_gcse[0] if len(hist_gcse)==1 else [None,None,None]

hist_A = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='történelem') & (df_erett_2020['szint']=='emelt')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
hist_A_pop,hist_A_perc,hist_A_avg = hist_A[0] if len(hist_A)==1 else [None,None,None]

eng_gcse = df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='angol nyelv') & (df_erett_2020['szint']=='közép')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
eng_gcse_pop,eng_gcse_perc,eng_gcse_avg = eng_gcse[0] if len(eng_gcse)==1 else [None,None,None]

eng_A= df_erett_2020[(df_erett_2020['érettségi vizsgatárgy']=='angol nyelv') & (df_erett_2020['szint']=='emelt')][['jelentkezők száma', 'százalékos átlag','osztályzat átlag']].values
eng_A_pop,eng_A_perc,eng_A_avg = eng_A[0] if len(eng_A)==1 else [None,None,None]


### Acceptence Rate

In [302]:
### Enrollment data
t_enroll2019 = soup.find('div', id='adat_kifir2019').find('table')

In [303]:
## Calcualte applied and accepted rates

enroll2019 = pd.read_html(str(t_enroll2019), header=0)[0]

first_place_acceptance_rate = enroll2019['1. helyen jelentkezettek közül a felvettek száma'].sum()/ enroll2019['Jelentkezők száma'].sum()
acceptance_rate = enroll2019['Felvettek száma'].sum() / enroll2019['Jelentkezők száma'].sum()

In [304]:
%who

BeautifulSoup	 acceptance_rate	 acceptence_rate	 address_id	 df_admin	 df_basics	 df_competency	 df_erett_2020	 df_students	 
df_teachers	 eng_A	 eng_A_avg	 eng_A_perc	 eng_A_pop	 eng_gcse	 eng_gcse_avg	 eng_gcse_perc	 eng_gcse_pop	 
enroll2019	 first_place_acceptance_rate	 first_place_acceptence_rate	 fulltime_teachers	 hist_A	 hist_A_avg	 hist_A_perc	 hist_A_pop	 hist_gcse	 
hist_gcse_avg	 hist_gcse_perc	 hist_gcse_pop	 hu_A	 hu_A_avg	 hu_A_perc	 hu_A_pop	 hu_gcse	 hu_gcse_avg	 
hu_gcse_perc	 hu_gcse_pop	 hu_gcse_res	 lit_competency	 math_A	 math_A_avg	 math_A_perc	 math_A_pop	 math_competency	 
math_gcse	 math_gcse_avg	 math_gcse_perc	 math_gcse_pop	 no_classes	 no_fail	 no_hs_student	 no_hs_student_female	 no_rooms	 
parttime_teachers	 pd	 requests	 res	 school_address	 school_dean	 school_dean_title	 school_id	 school_name	 
school_type	 soup	 t_2017	 t_2018	 t_2019	 t_2020	 t_basics	 t_enroll2019	 t_iq	 
t_students	 t_teachers	 test_table	 total_teachers	 


In [338]:
cols = ['school_id',
'school_name ',
'school_dean' ,
'school_dean_title',
'school_address',
'school_type',
'no_hs_student',
'no_hs_student_female' ,
'no_rooms ',
'no_classes',
'no_fail', 
'fulltime_teachers',
'parttime_teachers',
'total_teachers',
'first_place_acceptance_rate', 
'acceptance_rate',
'hu_gcse_pop','hu_gcse_perc','hu_gcse_avg',
'hu_A_pop','hu_A_perc','hu_A_avg',
'math_gcse_pop','math_gcse_perc','math_gcse_avg',
'math_A_pop','math_A_perc','math_A_avg', 
'hist_gcse_pop','hist_gcse_perc','hist_gcse_avg',
'hist_A_pop','hist_A_perc','hist_A_avg',
'eng_gcse_pop','eng_gcse_perc','eng_gcse_avg',
'eng_A_pop','eng_A_perc','eng_A_avg',
'math_competency','lit_competency']

In [341]:
pd.DataFrame(data=[school_id,
                    school_name ,
                    school_dean ,
                    school_dean_title,
                    school_address,
                    school_type,
                    no_hs_student,
                    no_hs_student_female ,
                    no_rooms ,
                    no_classes,
                    no_fail, 
                    fulltime_teachers,
                    parttime_teachers,
                    total_teachers,
                    first_place_acceptance_rate, 
                    acceptance_rate,
                    hu_gcse_pop,hu_gcse_perc,hu_gcse_avg,
                    hu_A_pop,hu_A_perc,hu_A_avg,
                    math_gcse_pop,math_gcse_perc,math_gcse_avg,
                    math_A_pop,math_A_perc,math_A_avg, 
                    hist_gcse_pop,hist_gcse_perc,hist_gcse_avg,
                    hist_A_pop,hist_A_perc,hist_A_avg,
                    eng_gcse_pop,eng_gcse_perc,eng_gcse_avg,
                    eng_A_pop,eng_A_perc,eng_A_avg,
                    math_competency,lit_competency
                    ],index=cols).T

,school_id,school_name,school_dean,school_dean_title,school_address,school_type,no_hs_student,no_hs_student_female,no_rooms,no_classes,...,hist_gcse_avg,hist_A_pop,hist_A_perc,hist_A_avg,eng_gcse_pop,eng_gcse_perc,eng_gcse_avg,eng_A_pop,eng_A_perc,eng_A_avg
0,035219,Budapest I. Kerületi Szilágyi Erzsébet Gimnázium,Bajkó Judit,intézményvezető,"1016 Budapest I. kerület, Mészáros utca 5-7.",[8 évfolyamos gimnáziumi nevelés-oktatás],501,256,21,16,...,4.65,8,73 %,4.88,19,88 %,4.89,25,77 %,4.76
